In [1]:
from bs4 import BeautifulSoup 
from urllib.request import urlopen

In [2]:
url_base = 'https://movie.naver.com'
url_sub = '/movie/bi/mi/point.nhn?code=182835'
html = urlopen(url_base + url_sub)
soup = BeautifulSoup(html, 'html.parser')

In [3]:
ifr_url = soup.find('iframe')['src']
ifr_url

'/movie/bi/mi/pointWriteFormList.nhn?code=182835&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false'

In [4]:
url = url_base + ifr_url
html = urlopen(url)
soup = BeautifulSoup(html, 'html.parser')

# 읽을 페이지 수 계산

In [5]:
count = soup.find('strong', 'total')
count

<strong class="total">관람객 평점 <em>1,856</em>건<button class="btn_review" id="open-form-btn">내 평점 등록</button></strong>

In [6]:
count = count.find('em').string     # get_text() 대신에 string
count

'1,856'

In [7]:
count = int(count.replace(',',''))
count

1856

In [8]:
import math
total_page = math.ceil(count/10)
total_page

186

In [9]:
point = []
review = []
date = []
for page in range(1, total_page+1):
    print(page, end=' ')
    url = url_base + ifr_url + '&page=' + str(page)
    html = urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')
    result = soup.find(class_='score_result')
    lis = result.find_all('li')
    for li in lis:
        score = li.find(class_='star_score').find('em').get_text()
        score = int(score)
        point.append(score)
        count_span = len(li.find(class_='score_reple').find(class_='').find_all('span'))
        count_icon = len(li.find_all(class_='ico_viewer'))
        count_spo = len(li.find_all(class_='text_spo'))
        criteria = str(count_span) + str(count_icon) + str(count_spo)
        #print(criteria)
        if criteria == '100':
            r = li.find(class_='score_reple').find('span').string.strip(' \t\r\n')
        elif criteria == '200':
            r = li.find(class_='score_reple').find('a')['data-src']
        elif criteria == '210':
            r = li.find(class_='score_reple').find_all('span')[1].string.strip(' \t\r\n')
        elif criteria == '310':
            r = li.find(class_='score_reple').find('a')['data-src']
        elif criteria == '311':
            r = li.find(class_='score_reple').find_all('span')[2].string.strip(' \t\r\n')
        elif criteria == '201':
            r = li.find(class_='score_reple').find_all('span')[1].string.strip(' \t\r\n')

        review.append(r)
        date.append(li.find(class_='score_reple').find_all('em')[1].get_text())

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 

In [13]:
import pandas as pd
df = pd.DataFrame({'평점':point, '일시':date, '감상평':review})
df

,평점,일시,감상평
0,2,2020.06.06 13:00,이상한데... 평점이 좋네. 왜지. 내가 이상한 사람인가
1,10,2020.06.04 12:49,오랜만에 극장에서 영화봐서 그런지 몰입감이 좋았습니다 송지효 목소리가 이렇게 소름돋...
2,6,2020.06.04 13:26,"초중반까지는 긴장감 좋고, 흥미롭다.그 후 텐션이 떨어지며 전개가 뻔해짐. 러닝타임..."
3,10,2020.06.04 11:53,송지효가 스릴러 연기를 이렇게 잘하는구나 느껴짐 .. 특유의 중저음 목소리랑 분위기...
4,10,2020.06.04 09:26,이거 진짜 재밌음 송지효랑 김무열 캐스팅 대박이다.
...,...,...,...
1851,10,2020.06.04 15:14,스포:송지효 죽음 ㅋㅋㅋㅋ
1852,4,2020.06.04 10:39,평식이형 5점이다 얘들아 댓글알바 알아서 잘거르고 요즘 개봉한 영화가 없어서 볼게 ...
1853,2,2020.06.04 09:50,9:22분에 조조 봤다는건 너무 알바 아니냐 ㅋㅋㅋ
1854,4,2020.06.04 14:42,런닝맨 송지효가 스릴러 영화의 주인공으로 나오는데 몰입이 될까? 저번에는 나혼자산다...


In [14]:
df.to_csv('data/intruder, 2019.csv')

In [15]:
len(df)

1856